## Code to read in state data

The data is from [COVID Tracking Project](https://covidtracking.com/).

This code is modified from [Jessica Nadalin].(https://github.com/jessicanada/covid_analysis/blob/master/StateByStateAnalysis.ipynb)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

## Get daily data from covidtracking.com

In [2]:
! rm states.csv
! wget https://covidtracking.com/api/states.csv --no-check-certificate

rm: states.csv: No such file or directory
--2020-04-10 14:23:08--  https://covidtracking.com/api/states.csv
Resolving covidtracking.com (covidtracking.com)... 159.65.216.232
Connecting to covidtracking.com (covidtracking.com)|159.65.216.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14087 (14K) [text/csv]
Saving to: ‘states.csv’

states.csv          100%[===================>]  13.76K  --.-KB/s    in 0.02s   

2020-04-10 14:23:08 (825 KB/s) - ‘states.csv’ saved [14087/14087]



In [3]:
! rm daily.csv
! wget http://covidtracking.com/api/states/daily.csv --no-check-certificate

--2020-04-10 14:23:09--  http://covidtracking.com/api/states/daily.csv
Resolving covidtracking.com (covidtracking.com)... 159.65.216.232
Connecting to covidtracking.com (covidtracking.com)|159.65.216.232|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246518 (241K) [text/csv]
Saving to: ‘daily.csv’

daily.csv           100%[===================>] 240.74K  --.-KB/s    in 0.1s    

2020-04-10 14:23:09 (1.73 MB/s) - ‘daily.csv’ saved [246518/246518]



In [4]:
states = pd.read_csv('states.csv')
daily = pd.read_csv('daily.csv')
daily['date']= pd.to_datetime(daily['date'],format='%Y%m%d')
daily['frac'] = daily['positive']/daily['total']
daily = daily[daily['state']!='AS']
# remove territories
states = states[states['state']!='AS']
states = states[states['state']!='MP']
states = states[states['state']!='GU']
states = states[states['state']!='PR']
states = states[states['state']!='VI']

## Save data for MATLAB

In [5]:
state = 'MA'        # State to load and save.
S = daily.copy()
S =S[S['state'] == state].sort_values('date')
S['cumpos'] = S['positive'].cumsum()
S = S[S['cumpos']>0].reset_index(drop=True)
I = S['positive']
date = S['date']
I0 = np.zeros(I.size)
d0 = np.zeros([I.size,3])
for k in np.arange(I.size):
    I0[k] = I[k]
    d0[k,:] = [date[k].month, date[k].day, date[k].year]
    
sio.savemat(state+'.mat', {'I':I0, 'd':d0})